In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
#from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
plt.figure(figsize=(10,10))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")


In [ ]:
len(dataset)

In [ ]:
80% ==> training
20% ==> 10% validation, 10% test

In [25]:
train_size = 0.8
len(dataset)*train_size

104.80000000000001

In [26]:
train_ds = dataset.take(104)
len(train_ds)

104

In [27]:
test_ds = dataset.skip(104)
len(test_ds)

27

In [28]:
val_size = 0.1
len(dataset)*val_size

13.100000000000001

In [29]:
val_ds = test_ds.take(6)
len(test_ds)

27

In [30]:
test_ds = test_ds.skip(6)
len(test_ds)

21

In [31]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)

    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [32]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [33]:
len(train_ds)

104

In [34]:
len(val_ds)

13

In [35]:
len(test_ds)

14

In [36]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [37]:
resize_and_rescale = tf.keras.Sequential([
  
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [38]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [40]:
# 1. Classes = 4 (Based on your first error)
n_classes = 4 

# 2. Input channels must be 3 (For RGB images)
# Your error shows your images are 500x500
IMAGE_SIZE = 500
CHANNELS = 3 

model = models.Sequential([
    # The '3' here represents RGB color channels
    layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    
    resize_and_rescale,
    data_augmentation,
    
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation= 'relu'),
    
    # This is where the '4' belongs
    layers.Dense(n_classes, activation= 'softmax'),
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
#model.compile(
   # optimizer='adam',
    #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    #metrics=['accuracy']
#)

In [ ]:
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data=val_ds
)

Epoch 1/50
 35/104 ━━━━━━━━━━━━━━━━━━━━ 1:17 1s/step - accuracy: 0.3030 - loss: 1.3579

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
scores

In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
history.history['accuracy']

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])

    batch_prediction = model.predict(images_batch)
    predicted_index = batch_prediction[0].argmax()
    print("predicted label:", class_names[predicted_index])

    

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array, verbose=0)

    predicted_index = int(tf.argmax(predictions[0]).numpy())
    predicted_class = class_names[predicted_index]
    confidence = round(float(tf.reduce_max(predictions[0]).numpy()) * 100, 2)

    return predicted_class, confidence


In [ ]:
plt.figure(figsize=(15, 15))

for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[int(labels[i])]

        plt.title(
            f"Actual: {actual_class}\n"
            f"Predicted: {predicted_class} ({confidence:.2f}%)\n"
            f"Confidence: {confidence}%"
        )
        plt.axis("off")


In [ ]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.export(f"../models/{model_version}")


In [ ]:
#model_version = 2
#model.export(f"../models/{model_version}")

In [ ]:
model.save("../models/corn_model.h5")
print("Model saved successfully!")

In [ ]:
import tensorflow as tf

# ඔයාගේ folder එකේ path එක (මෙතන '1' කියන folder එක ඇතුළේ model එක තියෙන නිසා)
# Path එක හරියටම පරීක්ෂා කරන්න
old_model_path = "../models/1" 

# Model එක load කරගන්න
temp_model = tf.keras.models.load_model(old_model_path)

# දැන් ඒක .h5 file එකක් විදිහට save කරන්න
temp_model.save("../models/corn_model.h5")

print("සාර්ථකව .h5 file එක හැදුනා!")

In [ ]:
import tensorflow as tf
import os

# 1. ඔයාගේ folder එක තිබුණ තැන (Screenshot එකේ හැටියට 'models/1' ඇතුළේ තමයි model එක තියෙන්නේ)
# මෙතන path එක ඔයාගේ පරිගණකයට අනුව නිවැරදිද බලන්න
model_folder_path = "../models/1"

if os.path.exists(model_folder_path):
    # 2. Folder එකෙන් model එක load කරනවා
    model = tf.keras.models.load_model(model_folder_path)
    
    # 3. ඒක තනි .h5 file එකක් විදිහට save කරනවා
    model.save("../models/corn_model.h5")
    print("සාර්ථකයි! දැන් 'models' folder එක ඇතුළේ 'corn_model.h5' file එක ඇති.")
else:
    print("Error: Folder එක සොයාගත නොහැක. Path එක පරීක්ෂා කරන්න.")